In [ ]:
import cv2
import mediapipe as mediapi
import math
from ctypes import cast,POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as nump

In [ ]:
capture = cv2.VideoCapture(0)
mediapiHands = mediapi.solutions.hands
fingures = mediapiHands.Hands()
dr = mediapi.solutions.drawing_utils
div = AudioUtilities.GetSpeakers()
infc = div.Activate(
IAudioEndpointVolume._iid_,CLSCTX_ALL, None)
vl = cast(infc, POINTER(IAudioEndpointVolume))

In [ ]:
while True:
    vic, i = capture.read()
    imageColour = cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
    rslt = fingures.process(imageColour)
    if rslt.multi_hand_landmarks:
        for fingL in rslt.multi_hand_landmarks:
            listL=[]
            for id,lm in enumerate(fingL.landmark):
                h,w,c = i.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                listL.append([id,cx,cy])
        if listL:
            x1,y1 = listL[4][1] , listL[4][2]
            x2,y2 = listL[8][1] , listL[8][2]
            cv2.circle(i,(x1,y1),10,(12,145,255), cv2.FILLED)
            cv2.circle(i,(x2,y2),10,(12,145,255), cv2.FILLED)
            cv2.line(i,(x1,y1),(x2,y2),(8,12,255),3)
            ln= math.hypot(x2-x1,y2-y1)
            if ln <50 :
                z1 = (x1+ x2)
                z2 = (y1+ y2)
                cv2.circle(i,(z1,z2),15,(2550,0,7),cv2.FILLED)
        vR = vl.GetVolumeRange()
        mnV = vR[0]
        mxV = vR[1]
        vol = nump.interp(ln , [50,300],[mnV,mxV])
        vB = nump.interp(ln, [50,300],[400,150])
        vl.SetMasterVolumeLevel(vol,None)
        cv2.rectangle(i,(50,150),(85,400),(254,8,0),3)
        cv2.rectangle(i,(50,int(vB)),(85,400),(0,124,234),cv2.FILLED)
    cv2.imshow("Image",i)
    cv2.waitKey(1)